# Data

In [ ]:
from nltk_postagger import pos_tag_fn,train_pos

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#df_h_s = pd.read_csv("data_hindi_sanskrit.csv")
df_h_s = pd.read_csv("encoded.csv")

In [ ]:
df_h_s

In [ ]:
df_h_s1 = df_h_s[:10]

In [ ]:
df_h_s1.to_csv("sample.csv")

In [ ]:
import transformers
from datasets import load_dataset, load_metric

In [ ]:
# dataset = load_dataset('csv', data_files="MKB_NIOS_data.csv", header=0, split='train')
# dataset = load_dataset('csv',data_files='sample.csv',header = 0, split='train')
dataset = load_dataset('csv',data_files="encoded.csv",header=0, split='train')

In [ ]:
dataset

In [ ]:
if 'Unnamed: 0' in dataset.column_names:
        dataset = dataset.remove_columns(['Unnamed: 0'])

In [ ]:
dataset = dataset.train_test_split(test_size=0.1, shuffle=False)

In [ ]:
dataset

# import modified model

In [ ]:
from interlingua_model.model.modelling_mbart_modified_token import MBartForConditionalGeneration

# later

In [ ]:
from transformers import (AutoTokenizer,
                          AutoModelForSeq2SeqLM,
                          #DataCollatorForSeq2Seq,
                          Seq2SeqTrainingArguments,
                          Seq2SeqTrainer)


## modified data collator:


In [ ]:
from interlingua_model.data_handling.datacollater import DataCollatorForSeq2Seq

In [ ]:
model_name = "ai4bharat/IndicBART"
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, use_fast=False, keep_accents=True)
#

In [ ]:
#model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path ="/home/ece/utkarsh/interlingua-model/IndicBART/trained-models")

model = MBartForConditionalGeneration.from_pretrained(pretrained_model_name_or_path = "/home/ece/utkarsh/interlingua-model/")

In [ ]:
#source_lang = 'devanagari_kannada' if FLAGS.source_lang.capitalize() == 'Kannada' else FLAGS.source_lang.capitalize()
#target_lang = 'devanagari_kannada' if FLAGS.target_lang.capitalize() == 'Kannada' else FLAGS.target_lang.capitalize()
source_lang = 'Hindi'
target_lang = 'Sanskrit'

In [ ]:
language_code = {'Hindi':'hi', 'Sanskrit':'sa', 'Kannada':'kn'}
#s_lang = language_code[FLAGS.source_lang.capitalize()]
#t_lang = language_code[FLAGS.target_lang.capitalize()]
s_lang = 'hi'
t_lang = 'sa'

In [ ]:
max_input_length = 128
max_target_length = 128

In [ ]:
from tqdm import tqdm

In [ ]:
def preprocess_function(examples):
        sentences= examples[source_lang]
        inputs = [example + ' </s>' + f' <2{s_lang}>' for example in examples[source_lang]]
        targets = [f'<2{t_lang}> ' + example + ' </s>' for example in examples[target_lang]]
        # print(inputs)
        # print(targets)
        model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=max_target_length, truncation=True)
        


        model_inputs['labels'] = labels['input_ids']
        print(type(labels))
        print(type(model_inputs))
        print(model_inputs.keys())
        print(len(inputs))
        lis_pos = []
        pos_tagger = train_pos()

        for i in tqdm(range(len(inputs))):
            # print(len(model_inputs['input_ids'][i]),model_inputs['input_ids'][i])
            # print
            pos_tag = pos_tag_fn(sentences[i],pos_tagger,tokenizer)
            # print("len",len(pos_tag),pos_tag)
            if len(model_inputs['input_ids'][i]) != len(pos_tag):
                # c+=1
                pos_update_val = []
                # update_pos = {}
                num = len(model_inputs['input_ids'][i]) - 2
                # print(i)
                pos_update_val += num * [6]
                pos_update_val.insert(0,2)
                pos_update_val.insert(len(pos_update_val),1)
                # print("Modified",len(pos_update_val),pos_update_val)
                # update_pos["pos"] = pos_update
                lis_pos.append(pos_update_val)
            else:
                # print('Original',len(pos_tag),pos_tag)
                lis_pos.append(pos_tag) 
        # for i in tqdm(range(len(inputs))):
        #     print(i)
        #     lis_pos.append(pos_tag_fn(inputs[i],pos_tagger,tokenizer))
            #print(i,val)
        model_inputs.data['pos'] = lis_pos
        # print(model_inputs)
        # model_inputs.concat(pos_dict)
        
        #a.append(model_inputs)
        return model_inputs


In [ ]:
"""Old preprocess function
def preprocess_function(examples):
        sentences= examples[source_lang]
        inputs = [example + ' </s>' + f' <2{s_lang}>' for example in examples[source_lang]]
        targets = [f'<2{t_lang}> ' + example + ' </s>' for example in examples[target_lang]]
        # print(inputs)
        # print(targets)
        model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=max_target_length, truncation=True)
        

        model_inputs['pos'] = examples['pos']
        model_inputs['labels'] = labels['input_ids']
        print(type(labels))
        print(type(model_inputs))
        print(model_inputs.keys())
        print(len(inputs))


        # print(examples['pos'])
        # model_inputs.concat(pos_dict)
        
        #a.append(model_inputs)
        return model_inputs
"""

# after exp

In [ ]:
batch_size = 16

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
#NIos + MKB--12K approx sentence-- 56mins


In [ ]:
len(tokenized_datasets["train"]['pos'][45])

In [ ]:
len(tokenized_datasets["train"]['input_ids'][45])

In [ ]:
len(tokenized_datasets['train']['pos'][45])

# tokenized_datasets.pickle

In [ ]:
import pickle

In [ ]:
filename = 'proc_tokenized_dataset_tensor_nios_mkb.pkl'
pickle.dump(tokenized_datasets, open(filename, 'wb'))

In [ ]:
rand = []
rand_sent = []
c = 0
for i in tokenized_datasets["train"]:
    if len(i["input_ids"]) != len(i["pos"]):
        # rand_sent.append(i['Hindi'])
        # temp = [i['Hindi'],i['pos'],i['input_ids']]
        # rand.append(temp)
        c +=1
        # rand
print(c)